In [145]:
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import math
import random
import os
import pandas as pd

In [60]:
import sys
DATASET_DIR = "../datasets/"

In [87]:
df_ann = pd.DataFrame(columns=["image", "mask"])

In [88]:
df_ann

,image,mask


In [79]:
import warnings
warnings.filterwarnings('ignore')

In [136]:
image_names = list(Path(DATASET_DIR + "Pages").glob("*"))

In [140]:
# for img_path in Path(DATASET_DIR + "tables").glob("*"):
# image = cv2.imread("/home/cseadmin/Tigran/table_extractor/datasets/Pages/001.pdf_page_0.jpg", cv2.IMREAD_GRAYSCALE)
image = cv2.imread(str(image_names[2]), cv2.IMREAD_GRAYSCALE)

# Use canny edge detection
edges = cv2.Canny(image, 50, 200, None, 3)

print(edges.min(), edges.max())
Image.fromarray(edges).show()

0 255


In [143]:
table_names = os.listdir(DATASET_DIR + "tables")
pdf_page_names = os.listdir(DATASET_DIR + "Pages")

In [154]:
batch_X, batch_y = [], []
while True:

    random_table_name = table_names[random.randint(0, len(table_names))]
    random_page_name = pdf_page_names[random.randint(0, len(pdf_page_names))]

    table_img = cv2.imread("../datasets/tables/" + random_table_name, cv2.IMREAD_GRAYSCALE)
    page_img = cv2.imread("../datasets/Pages/" + random_page_name, cv2.IMREAD_GRAYSCALE)

    current_possible_pos = []
    if table_img.shape[0] > page_img.shape[0] or table_img.shape[1] > page_img.shape[1]:
        continue

    for i in range(0, page_img.shape[0] - table_img.shape[0], 10):
        for j in range(0, page_img.shape[1] - table_img.shape[1], 10):
            if np.all(page_img[i:i+table_img.shape[0], j:j+table_img.shape[1]] == 255):
                current_possible_pos.append((i, j))

    if len(current_possible_pos) > 0:
        rand_ind = np.random.choice(np.arange(len(current_possible_pos)))
        y, x = current_possible_pos[rand_ind]
        page_img[y:y+table_img.shape[0], x:x+table_img.shape[1]] = table_img
        batch_X.append(page_img)
        mask = np.zeros_like(page_img)
        mask[y:y+table_img.shape[0], x:x+table_img.shape[1]] = 255
        batch_y.append(mask)

    if len(batch_X) == 2:
        break

In [155]:
Image.fromarray(batch_X[1]).show()
Image.fromarray(batch_y[1]).show()

In [86]:
for i, table_img_path in enumerate(Path(DATASET_DIR + "tables").glob("*")):
    table_img = cv2.imread(str(table_img_path), cv2.IMREAD_GRAYSCALE)
    table_bn, ex = os.path.splitext(table_img_path.name)
    if (table_img.shape[0] < 600 and table_img.shape[1] < 600):
        continue
    # ran = np.arange(0.9, 1.1, 0.05)
    # fig, ax = plt.subplots(len(ran), 1, figsize=(40, 40))
    # for i, scale in enumerate(ran):
    #     scaled_h, scaled_w = int(table_img.shape[0] * scale), int(table_img.shape[1] * scale)
    #     scaled_img = cv2.resize(table_img, (scaled_h, scaled_w), cv2.INTER_AREA)
        # ax[i].imshow(scaled_img)
    for page_path in Path(DATASET_DIR + "Pages").glob("*"):
        page_img = cv2.imread(str(page_path), cv2.IMREAD_GRAYSCALE)
        if page_img.shape[0]<table_img.shape[0] or page_img.shape[1]<table_img.shape[1]: continue
        temp = np.ones((table_img.shape[0], table_img.shape[1])).astype(np.uint8) * 255
        res = cv2.matchTemplate(page_img, temp, method=cv2.TM_CCORR_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        start_x, start_y = max_loc
        end_x, end_y = (start_x + temp.shape[1], start_y + temp.shape[0])
        # print(start_x, start_y, end_x, end_y)
        if not np.all(page_img[start_y:end_y, start_x:end_x] == 255): continue
        page_img[start_y:end_y, start_x:end_x] = table_img
        # Image.fromarray(page_img).show()
        mask = np.zeros_like(page_img)
        mask[start_y:end_y, start_x:end_x] = 255
        b_n, ext = os.path.splitext(page_path.name)
        Image.fromarray(page_img).save(f"{DATASET_DIR}synthetic_images/{b_n}_{table_bn}.jpg")
        df_ann = df_ann.append(
            {"image" : f"{b_n}_{table_bn}.jpg", "mask":[start_x, end_x, start_y, end_y]},
            ignore_index=True
        )
        # Image.fromarray(mask).save(f"{DATASET_DIR}synthetic_masks/{b_n}_{table_bn}_mask.png")
        print(f"Saved {b_n}_{table_bn} - {i+1}/{len(os.listdir(DATASET_DIR + 'tables'))}", end='\r')
        # break

    # break

KeyboardInterrupt: 

In [71]:
df_ann

,image,mask
0,Abdulaziz-Bayindir-Kuran-Isiginda-tarikatcilig...,"[295, 932, 14, 170]"


In [33]:
W, H = 1600, 700

for page_path in Path(DATASET_DIR + "Pages").glob("*"):
    page_img = cv2.imread(str(page_path), cv2.IMREAD_GRAYSCALE)
    temp = np.ones((H, W)).astype(np.uint8) * 255
    res = cv2.matchTemplate(page_img, temp, method=cv2.TM_CCORR_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    start_x, start_y = max_loc
    end_x, end_y = (start_x + temp.shape[1], start_y + temp.shape[0])
    print(start_x, start_y, end_x, end_y)
    page_img[start_y:end_y, start_x:end_x] = temp
    # page_img = cv2.rectangle(page_img, (start_x, start_y), (end_x, end_y), 0,5)

    Image.fromarray(page_img).show()
    break

0 1442 1600 2142
